In [1]:
# 필요한 라이브러리 불러오기
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time
import pandas as pd
import numpy as np

# 피파 랭킹 순위 데이터 가져오기

In [2]:
#  피파 랭킹 순위 데이터 가져오기
url_base = 'http://data.7m.com.cn/fifarank/index_kr.aspx'
page = urlopen(url_base)
soup = BeautifulSoup(page,'html.parser')

In [3]:
soup


<script>

function Search_Click(){
	//location.href = "index_kr.aspx?type=" 
	//	+ gType.value + "&d=" + gYear.value 
	//	+ "-" + gMonth.value;
    var gtv = document.getElementById("gType").value;
	var ytv = document.getElementById("gYear").value;
    var mtv = document.getElementById("gMonth").value;
    location.href = "index_kr.aspx?type="
        + gtv + "&d=" + ytv
        + "-" + mtv;
}
</script>
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">

<html>
<head>
<meta content="zh-cn" http-equiv="Content-Language"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>남자축구 FIFA월드랭킹 – 7M스포츠</title>
<meta content="라이브스코어, 축구스코어, 스코어, 농구스코어, 테니스스코어, 대진분석, 순위표, 득점선수순위, 축구데이터, 농구데이터, FIFA랭킹" name="keywords"/>
<link href="style.css" rel="stylesheet" type="text/css"/>
<script language="javascript" src="/js/utf-8/Open_Link.js"></script>
</head>
<body>
<form action="./index_kr.aspx" id="ctl00" method="post">
<input id="__VIEWSTATE" name="__VIEWSTATE" ty

In [4]:
#rankData > table:nth-child(2) > tbody > tr:nth-child(2) > td:nth-child(1)- 랭킹1위
#rankData > table:nth-child(2) > tbody > tr:nth-child(3) > td:nth-child(1)- 랭킹2위
#rankData > table:nth-child(2) > tbody > tr:nth-child(217) > td:nth-child(1) - 랭킹 꼴찌

In [5]:
# 타겟 설정하기
soup.select('span#rankData > table > tr:nth-of-type(2) > td')

[<td>1</td>,
 <td><a href="javascript:Country_kr(65)"><img border="0" src="/team_data/481/logo_img/flag.gif"/></a></td>,
 <td><a href="javascript:Team_kr(481)">벨기에</a></td>,
 <td>0<font color="#336699" face="Marlett">34</font></td>,
 <td>1727</td>,
 <td>0<font color="#336699" face="Marlett">34</font></td>,
 <td><a href="chart.htm?ecd=5&amp;id=74" target="_blank"><img border="0" src="http://img.7m.com.cn/img10//i_rs1.gif"/></a></td>]

In [6]:
# 포멧 살펴보기
target = soup.select('span#rankData > table > tr:nth-of-type(2) > td')

In [7]:
for i in target:
    print(i.string)


1
None
벨기에
None
1727
None
None


In [8]:
# 출력값 데이터화 시키기
rankall = []

for k in range(2,218):
    target = soup.select('span#rankData > table > tr:nth-of-type(%d) > td'%(k))
    rank = []
    for i in target:
        rank.append(i.string)
    rankall.append(rank)

In [9]:
# rankall 데이터화시키기
FIFARANK = pd.DataFrame(rankall)
FIFARANK.head()

,0,1,2,3,4,5,6
0,1,None,벨기에,None,1727,None,None
1,2,None,프랑스,None,1726,None,None
2,3,None,브라질,None,1676,None,None
3,4,None,크로아티아,None,1634,None,None
4,5,None,잉글랜드,None,1631,None,None


In [10]:
# 랭킹 데이터 정제하기 - 중간에 있는 칼럼 데이터 삭제
# 위치 확인
# 모든 랭킹 값 가져오기
# print(FIFARANK.loc[:,0])
# 랭킹값에 문자로 써있는 부분 추출 null값으로 삽입
FIFARANK.loc[FIFARANK[0]=='랭킹',0] = np.nan
# 필요없는 칼럼 지우기
FIFARANK = FIFARANK[[0,2,4]]
# null값이 포함된 행 지우기
FIFARANKdrop = FIFARANK.dropna(how='any')
# 데이터 확인이 불가 - 일단 데이터 프레임 정리 후 저장해보기
FIFARANKdrop.rename(columns={
    0 : '랭킹',
    2 : '국가',
    4 : '점수'},inplace = True)

FIFARANKdrop.head()

# 인덱스 변경


/Users/jooyeon/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,랭킹,국가,점수
0,1,벨기에,1727
1,2,프랑스,1726
2,3,브라질,1676
3,4,크로아티아,1634
4,5,잉글랜드,1631


In [11]:
FIFARANKdrop.columns

Index(['랭킹', '국가', '점수'], dtype='object')

In [12]:
# 데이터 저장하기
# FIFARANKdrop.to_csv('./data/FIFARank.csv',index=False)

In [13]:
pd.read_csv('./data/FIFARANK.csv')

,랭킹,국가,점수
0,1,벨기에,1727
1,2,프랑스,1726
2,3,브라질,1676
3,4,크로아티아,1634
4,5,잉글랜드,1631
5,6,포르투갈,1614
6,7,우루과이,1609
7,8,스위스,1599
8,9,스페인,1591
9,10,덴마크,1589


# 감독이름 Scrapy 하기

구글 드라이버를 이용해서 네이버에 검색하여 감독이름 수집하기


In [14]:
# 참가국
AsianCupList = ['대한민국', '레바논', '바레인', '베트남', '북한', '사우디아라비아', '시리아', '아랍에미레이트', \
'예멘', '오만', '요르단', '우즈베키스탄', '이라크', '이란', '인도', '일본', \
'중국', '카타르', '키르기스스탄', '태국', '투르크메니스탄', '팔레스타인', '필리핀', '호주' ]
AsianCupList

['대한민국',
 '레바논',
 '바레인',
 '베트남',
 '북한',
 '사우디아라비아',
 '시리아',
 '아랍에미레이트',
 '예멘',
 '오만',
 '요르단',
 '우즈베키스탄',
 '이라크',
 '이란',
 '인도',
 '일본',
 '중국',
 '카타르',
 '키르기스스탄',
 '태국',
 '투르크메니스탄',
 '팔레스타인',
 '필리핀',
 '호주']

In [15]:
# 구글 드라이버 사용
from selenium import webdriver
driver = webdriver.Chrome('./chromedriver.exe')
driver.get('https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%ED%98%B8%EC%A3%BC+%EC%B6%95%EA%B5%AC+%EA%B0%90%EB%8F%85&oquery=UAE+%EC%95%84%EC%8B%9C%EC%95%88%EC%BB%B5+%EC%B0%B8%EA%B0%80%EA%B5%AD&tqi=UbviFlpySoCsscyQS7KssssstVl-018601')

WebDriverException: Message: 'chromedriver.exe' executable may have wrong permissions. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [ ]:
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')
soup.select('tr:nth-of-type(1) > td:nth-of-type(2) > p > a')

In [ ]:
# soup 살펴보기
soup

In [ ]:
target = soup.select('tr:nth-of-type(1) > td:nth-of-type(2) > p > a')

In [ ]:
# test
for i in target:
    print(i.string)

In [ ]:
# 검색해서 가져오는 for 문 만들기
# 국가 감독명 리스트
NatDB = []
for k in AsianCupList:
    time.sleep(1)
    # 검색 지우기 클릭 입력
    xpath = '//*[@id="nx_query"]'
    driver.find_element_by_xpath(xpath).clear()
    driver.find_element_by_xpath(xpath).send_keys(k+" 축구 감독")
    xpath = '//*[@id="nx_search_form"]/fieldset/button/span'
    driver.find_element_by_xpath(xpath).click()
    time.sleep(3)
    # 페이지 안에서 데이터 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    target= soup.select('tr:nth-of-type(1) > td:nth-of-type(2) > p > a')
    # 타겟 리스트에 저장
    NatD = []
    for i in target:
        NatD.append(i.string)
    NatDB.append(NatD)
    print(k,NatD)
    time.sleep(1)


In [ ]:
# 데이터 프레임으로 만들기
NAT=pd.DataFrame(AsianCupList)
DIC=pd.DataFrame(NatDB)
NATDIC = pd.merge(NAT, DIC, on=NAT.index)
# 칼럼명 변경
NATDIC.columns = ['index', '국가', '감독']
NATDIC.head()

In [ ]:
# 데이터 저장
NATDIC.to_csv('./data/NATDir.csv',index=False)

# 대진표 Scrapy

In [ ]:
#  대진표 데이터 가져오기
url_base = 'https://www.scoreboard.com/kr/soccer/asia/asian-cup/results/'
page = urlopen(url_base)
soup = BeautifulSoup(page,'html.parser')

In [ ]:
# 데이터 살펴보기
soup

In [ ]:
1
//*[@id="g_1_xt2NNPcI"]/td[2]
//*[@id="g_1_xt2NNPcI"]/td[3]/span 
//*[@id="g_1_xt2NNPcI"]/td[4]/span
//*[@id="g_1_xt2NNPcI"]/td[5]
2
//*[@id="g_1_I9vaY8EJ"]/td[2]
3
//*[@id="g_1_ze8NxJdj"]/td[2]
4
//*[@id="g_1_G8zHJYuh"]/td[2]
5
//*[@id="g_1_jkBMX2Mo"]/td[2]
6
//*[@id="g_1_tAlLr29r"]/td[2]
7
//*[@id="g_1_CznCcKmG"]/td[2]
8
//*[@id="g_1_l8JFEKfQ"]/td[2]
9
//*[@id="g_1_6qUAF0uK"]/td[2]
10
//*[@id="g_1_pruaNK2H"]/td[2]
11
//*[@id="g_1_vBF7EawT"]/td[2]
12
//*[@id="g_1_ShV6GtQD"]/td[2]
13
//*[@id="g_1_MerLhwY6"]/td[2]
14
//*[@id="g_1_2JkfO0mB"]/td[2]
15
//*[@id="g_1_0nsPicmD"]/td[2]

//*[@id="g_1_xt2NNPcI"]/td[2]

In [ ]:
# 아이디 값 가져오기
for i in soup.select('tr'):
    print(i)

In [ ]:
xpath = '//*[@id="g_1_xt2NNPcI"]/td[2]'
driver.find_element_by_xpath(xpath).text

In [ ]:
xpath = '//*[@id="g_1_xt2NNPcI"]/td[3]/span '
driver.find_element_by_xpath(xpath).text

In [ ]:
xpath = '//*[@id="g_1_xt2NNPcI"]/td[4]/span'
driver.find_element_by_xpath(xpath).text

In [ ]:
xpath = '//*[@id="g_1_xt2NNPcI"]/td[5]'
driver.find_element_by_xpath(xpath).text

In [ ]:
# 첫줄은 다 가져왔다. 다음줄 부터 어떻게 변경되나 확인

In [ ]:
# 구글 드라이버 사용
from selenium import webdriver
driver = webdriver.Chrome('./chromedriver.exe')
driver.get('https://www.scoreboard.com/kr/soccer/asia/asian-cup/results/')

## xpath가 바뀐다. 아이디만 바뀌고 나머지는 그대로
# xpath 가지고 왔고 이제 xpath기반으로 for 문 만들기
XPATH = ['"g_1_xt2NNPcI"','"g_1_I9vaY8EJ"','"g_1_ze8NxJdj"','"g_1_G8zHJYuh"','"g_1_jkBMX2Mo"','"g_1_tAlLr29r"','"g_1_CznCcKmG"',\
         '"g_1_l8JFEKfQ"','"g_1_6qUAF0uK"','"g_1_pruaNK2H"','"g_1_vBF7EawT"','"g_1_ShV6GtQD"','"g_1_MerLhwY6"','"g_1_2JkfO0mB"',\
         '"g_1_0nsPicmD"']
Match = []
for i in XPATH:
    oneM = []
    xpath = '//*[@id=%s]/td[2]'%i
    a = driver.find_element_by_xpath(xpath).text
    oneM.append(a)
    xpath = '//*[@id=%s]/td[3]/span'%i
    a = driver.find_element_by_xpath(xpath).text
    oneM.append(a)
    xpath = '//*[@id=%s]/td[4]/span'%i
    a = driver.find_element_by_xpath(xpath).text
    oneM.append(a)
    xpath = '//*[@id=%s]/td[5]'%i
    a = driver.find_element_by_xpath(xpath).text
    oneM.append(a)
    Match.append(oneM)
    
Match

In [ ]:
Match=pd.DataFrame(Match)
Match

In [ ]:
Match.to_csv('./data/Match_playoff.csv',index=False)

In [ ]:
Match = pd.read_csv('./data/Match_playoff.csv')
Match

In [ ]:
Match = Match[['경기날짜', '팀1', '팀2', '점수']]

In [ ]:
socre = []
for i in Match['점수']:
    socre.append(i.split(':'))
socre = pd.DataFrame(socre)

In [ ]:
socre

In [ ]:
MatchA = pd.merge(Match,socre, left_on=Match.index, right_on=socre.index)
MatchA.head()

In [ ]:
MatchB=MatchA[['경기날짜', '팀1', '팀2', 0, 1]]
MatchC=MatchA[['경기날짜', '팀1', '팀2', 0, 1]]
MatchB.columns=['경기날짜', '팀1', '팀2', '팀1점수', '팀2점수']
MatchC.columns=['경기날짜', '팀1', '팀2', '팀1점수', '팀2점수']
MatchA

In [ ]:
# 팀 1 점수 팀 2 점수로 승패 나타내기 - 정수로 변경 비교 칼럼 넣기
MatchB['팀1점수']=MatchB['팀1점수'].astype('int')
MatchB['팀2점수']=MatchB['팀2점수'].astype('int')
# 승패 점수 넣기
MatchB.loc[MatchB['팀1점수'] > MatchB['팀2점수'],'승패'] = np.int(2)
MatchB.loc[MatchB['팀1점수'] < MatchB['팀2점수'],'승패'] = np.int(0)
MatchB.loc[MatchB['팀1점수'] == MatchB['팀2점수'],'승패'] = np.int(1)
# 절댓값 점수차 넣기
MatchB['점수차'] = (MatchB['팀1점수']-MatchB['팀2점수'])

In [ ]:
# 팀 1 점수 팀 2 점수로 승패 나타내기 - 정수로 변경 비교 칼럼 넣기
MatchC['팀1점수']=MatchC['팀1점수'].astype('int')
MatchC['팀2점수']=MatchC['팀2점수'].astype('int')
# 승패 점수 넣기
MatchC.loc[MatchB['팀1점수'] < MatchC['팀2점수'],'승패'] = np.int(2)
MatchC.loc[MatchB['팀1점수'] > MatchC['팀2점수'],'승패'] = np.int(0)
MatchC.loc[MatchB['팀1점수'] == MatchC['팀2점수'],'승패'] = np.int(1)
# 절댓값 점수차 넣기
MatchC['점수차'] = (MatchC['팀2점수']-MatchC['팀1점수'])

In [ ]:
MatchB=MatchB[['경기날짜', '팀1', '팀1점수', '팀2점수', '승패', '점수차']]
MatchB.head()

In [ ]:
MatchC=MatchC[['경기날짜', '팀2', '팀1점수', '팀2점수', '승패', '점수차']]
MatchC.head()

In [ ]:
MatchB.columns=['경기날짜', '팀', '팀1점수', '팀2점수', '승패', '점수차']
MatchC.columns=['경기날짜', '팀', '팀1점수', '팀2점수', '승패', '점수차']

In [ ]:
Mathcomp = pd.concat([MatchB,MatchC])
Mathcomp['승패']=Mathcomp['승패'].astype('int')
Mathcomp

In [ ]:
MatchD = pd.read_csv('./data/19year_ver_1_190211.csv')

In [ ]:
MatchComp19 = pd.concat([Mathcomp,MatchD])
MatchComp19

In [ ]:
MatchComp19.to_csv('./data/AsianCup19.csv', encoding='utf-8', index=False)

In [ ]:
players=pd.read_csv('./total_players.csv')
players.head()

In [ ]:
players.pivot_table('cap', index='nation',aggfunc=np.sum)